Comparison between Parcollet QNN (P-QNN) and this library (QNN), explicitly writing out the operation of a Linear layer with 1 input and 1 output in both libraries.

In [73]:
# Main PyTorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F

In [74]:
# Import P-QNN in path
import sys
sys.path.append("../Pytorch-Quaternion-Neural-Networks/core_qnn")

In [75]:
# P-QNN import (this is the function used in the forward pass of QuaternionLinearAutograd)
from quaternion_ops import quaternion_linear

In [76]:
# QNN imports (we only use the basic class here)
from quaternion import Quaternion

In [77]:
def init_values():
    # "Input": 0.1 + 0.2i + 0.3j + 0.4k
    x = torch.FloatTensor([[0.1, 0.2, 0.3, 0.4]])

    # "Weight": 1.0 + 0.9i + 0.8j + 0.7k
    r = torch.FloatTensor([[1.0]]).requires_grad_(True)
    i = torch.FloatTensor([[0.9]]).requires_grad_(True)
    j = torch.FloatTensor([[0.8]]).requires_grad_(True)
    k = torch.FloatTensor([[0.7]]).requires_grad_(True)

    return x, (r, i, j, k)

In [78]:
# Multiplication in P-QNN
x, (r, i, j, k) = init_values()
out_pqnn = quaternion_linear(x, r, i, j, k, bias=None)

In [80]:
# Gradient in P-QNN
(g_r_pqnn, g_i_pqnn, g_j_pqnn, g_k_pqnn) = torch.autograd.grad(out_pqnn.sum(), [r, i, j, k])

In [81]:
# Multiplication in QNN
x, (r, i, j, k) = init_values()
w = Quaternion(torch.cat([r, i, j, k], dim=1))
out_qnn = F.linear(x, w.real_repr.torch(), None)

In [83]:
(g_r_qnn, g_i_qnn, g_j_qnn, g_k_qnn) = torch.autograd.grad(out_qnn.sum(), [r, i, j, k])

In [85]:
print(out_pqnn)
print(out_qnn)

tensor([[-0.6000,  0.4000,  0.1600,  0.5800]], grad_fn=<MmBackward>)
tensor([[-0.6000,  0.4000,  0.1600,  0.5800]], grad_fn=<MmBackward>)


In [86]:
print(g_r_pqnn)
print(g_r_qnn)

tensor([[1.]])
tensor([[1.]])


In [87]:
print(g_i_pqnn)
print(g_i_qnn)

tensor([[-0.2000]])
tensor([[-0.2000]])


In [88]:
print(g_j_pqnn)
print(g_j_qnn)

tensor([[0.]])
tensor([[0.]])


In [89]:
print(g_k_pqnn)
print(g_k_qnn)

tensor([[-0.4000]])
tensor([[-0.4000]])
